# Manipulation d'images satellite avec SNAP et la librairie snapista

In [1]:
from pathlib import Path

import geopandas as gpd
import osmnx as ox
import pandas as pd
import rasterio
import rasterio.features
from shapely.geometry import Point, Polygon
from snapista import Graph, Operator, TargetBand, TargetBandDescriptors, graph_io

INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: GDAL not found on system. Internal GDAL 3.0.0 from distribution will be used. (f1)
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.


On peut récupérer la liste de tous les opérateurs disponibles :

In [2]:
Graph.describe_operators()

PssraOp - Pigment Specific Simple Ratio, chlorophyll index
Calibration - Calibration of products
MtciOp - The Meris Terrestrial Chlorophyll Index,  aims at estimating the Red Edge Position (REP).
This is the maximum slant point in the red and near-infrared region of the vegetal spectral reflectance.
It is useful for observing the chlorophyll contents, vegetation senescence, and stress for water and nutritional deficiencies, but it is less suitable for land classification
StatisticsOp - Computes statistics for an arbitrary number of source products.
Multi-Temporal-Speckle-Filter - Speckle Reduction using Multitemporal Filtering
Meris.AerosolMerger - None
Cross-Correlation - Automatic Selection of Ground Control Points
IreciOp - Inverted red-edge chlorophyll index
Meris.CorrectRadiometry - Performs radiometric corrections on MERIS L1b data products.
CoregistrationOp - Coregisters two rasters, not considering their location
BandsDifferenceOp - None
ReflectanceToRadianceOp - The 'Reflectan

{'PssraOp': 'Pigment Specific Simple Ratio, chlorophyll index',
 'Calibration': 'Calibration of products',
 'MtciOp': 'The Meris Terrestrial Chlorophyll Index,  aims at estimating the Red Edge Position (REP).\nThis is the maximum slant point in the red and near-infrared region of the vegetal spectral reflectance.\nIt is useful for observing the chlorophyll contents, vegetation senescence, and stress for water and nutritional deficiencies, but it is less suitable for land classification',
 'StatisticsOp': 'Computes statistics for an arbitrary number of source products.',
 'Multi-Temporal-Speckle-Filter': 'Speckle Reduction using Multitemporal Filtering',
 'Meris.AerosolMerger': None,
 'Cross-Correlation': 'Automatic Selection of Ground Control Points',
 'IreciOp': 'Inverted red-edge chlorophyll index',
 'Meris.CorrectRadiometry': 'Performs radiometric corrections on MERIS L1b data products.',
 'CoregistrationOp': 'Coregisters two rasters, not considering their location',
 'BandsDifferen

On peut également avoir la description d'un opérateur. Pour cela, on instancie d'abord un objet `Operator`.

In [3]:
read_op = Operator('Read')

In [4]:
read_op.describe()

INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.


Operator name: Read

Description: Reads a data product from a given file location.
Authors: Marco Zuehlke, Norman Fomferra

org.esa.snap.core.gpf.common.ReadOp
Version: 1.2

Parameters:

	file: The file from which the data product is read.
		Default Value: None

		Possible values: []

	formatName: An (optional) format name.
		Default Value: None

		Possible values: ['SENTINEL-2-MSI-MultiRes-UTM07N', 'SeaDAS-Browse', 'AVNIR-2', 'NOAA_AVHRR_3_L1B', 'SPOT-VGT', 'BEAM-DIMAP', 'SENTINEL-2-MSI-MultiRes-UTM38N', 'SENTINEL-2-MSI-MultiRes-UTM47N', 'AlosAV2Dimap', 'Basic CEOS', 'MER_L2_S3', 'SENTINEL-2-MSI-MultiRes-UTM49S', 'SENTINEL-2-MSI-MultiRes-UTM04S', 'SENTINEL-2-MSI-MultiRes-UTM26N', 'SENTINEL-2-MSI-MultiRes-UTM35N', 'GeoTIFF-BigTIFF', 'SENTINEL-2-MSI-MultiRes-UTM23N', 'SeaDAS-ANC', 'SENTINEL-2-MSI-MultiRes-UTM02N', 'SENTINEL-2-MSI-MultiRes-UTM10N', 'SENTINEL-2-MSI-MultiRes-UTM53N', 'OCM2-L1B', 'Binned_data_product', 'SENTINEL-2-MSI-MultiRes-UTM35S', 'PACE-L1B', 'SENTINEL-2-MSI-MultiRes-U

In [5]:
sat_img_path = './data/S2A_MSIL1C_20210415T154901_N0300_R054_T18TYT_20210417T135102.SAFE.zip'
sat_img_name = f'{Path(sat_img_path).stem.split(".")[0]}'

On peut maintenant définir l'objet `read_op`.

In [6]:
read_op.file = sat_img_path

On va maintenant commencer à analyser l'image `S2A_MSIL1C_20210415T154901_N0300_R054_T18TYT_20210417T135102.SAFE.zip` à l'aide de SNAP. Cette image couvre notamment le [Lac Saint-Charles](https://fr.wikipedia.org/wiki/Lac_Saint-Charles) situé au Canada. Nous allons mettre en place un algorithme permettant d'extraire la quantité de chlorophylle-a pour chaque pixel de l'image appartenant à ce lac.

La première étape consiste à faire un `resampling` de l'image, c'est-à-dire faire en sorte que toutes les bandes de l'image multi-spectrale aient la même résolution. Pour cela, il existe un opérateur `Resample`.

On va alors lui préciser la résolution souhaitée en mètres (ici 30 mètres) ainsi que la méthode de resampling que l'on souhaite utiliser (ici bilinéaire).

In [7]:
resample_op = Operator('Resample')
resample_op.targetResolution = '60'  # il faut que ça soit une chaîne de caractère et non un type numérique
resample_op.upsamplingMethod = 'Bilinear'

Les images Sentinel-2 couvrent une très grande zone géographique (environ $110 \times 110~km^2$). Pour réduire les temps de traitement, nous allons cropper notre image autour de cette zone.

La première étape consiste à récupérer le polygone du lac. Pour cela, nous utilisons les données [OpenStreetMap](https://www.openstreetmap.org/#map=14/46.94587/-71.38972) que nous requêtons grâce à la librairie python [OSMnx](https://osmnx.readthedocs.io/en/stable/). Nous avons besoin uniquement des coordonnées (latitude et longitude) d'un point du lac.

In [8]:
lat, lon = 46.94, -71.39
gdf_lake = ox.geometries_from_point((lat, lon), tags={'water': 'lake'}).reset_index(drop=True)[['geometry']]
geom_col = gdf_lake.geometry.name

# on rajoute un buffer autour du polygone du lac
crs = gdf_lake.crs
gdf_lake.to_crs('EPSG:3857', inplace=True)
gdf_lake['buffered_polygon'] = gdf_lake['geometry'].buffer(45)

for col in [geom_col, 'buffered_polygon']:
    gdf_lake.set_geometry(col, inplace=True)
    gdf_lake.to_crs(crs, inplace=True)
    
lake_polygon = gdf_lake['buffered_polygon'][0]
gdf_lake.set_geometry(geom_col, inplace=True)
gdf_lake.drop(columns='buffered_polygon', inplace=True)

On peut maintenant réaliser le crop à l'aide de l'opérateur `Subset`.

In [9]:
subset_op = Operator('Subset')
subset_op.geoRegion = lake_polygon.envelope.wkt  # get string representation of the polygon envelope
subset_op.copyMetadata = "true"
subset_op.tiePointGridNames = ','  # see https://forum.step.esa.int/t/subset-error-gridheight-2/34577/6

On utilise maintenant l'outil [C2RCC](https://c2rcc.org/) pour extraire la quantité de chlorophylle-a.

In [10]:
c2rcc_op = Operator('c2rcc.msi')
c2rcc_op.validPixelExpression = 'B8 &gt; 0 &amp;&amp; B8 &lt; 0.4'
c2rcc_op.salinity = '2.81e-5'  # min value is 2.8e-5, excluded
c2rcc_op.netSet = 'C2X-COMPLEX-Nets'

Afin de pouvoir sélectionner puis exporter dans un fichier les bandes nous intéressant, nous devons d'abord les convertir (elles sont actuellement considérées comme des bandes "virtuelles").

In [11]:
bands = ['x', 'y', 'lat', 'lon', 'c2rcc_flags', 'conc_chl', 'unc_chl']

In [12]:
bands_op = {}
for band in bands:
    expr = band if band in ['c2rcc_flags', 'conc_chl', 'unc_chl'] else band.upper()
    type = 'uint16' if band in ['x', 'y'] else 'float32'
    band_op = Operator('BandMaths')
    band_op.targetBandDescriptors = TargetBandDescriptors([TargetBand(name=band, expression=expr, type=type)])
    bands_op[f'{band}_band'] = band_op

Il faut maintenant créer un produit en "fusionnant" toutes ces bandes à l'aide de l'opérateur `BandMerge`.

On sélectionne maintenant les bandes que l'on souhaite exporter, encore à l'aide de l'opérateur `Subset`.

In [13]:
band_merge_op = Operator('BandMerge')

In [14]:
band_subset_op = Operator('Subset')
band_subset_op.bandNames = ','.join(bands)
band_subset_op.tiePointGridNames = ','
band_subset_op.copyMetadata = 'false'

Il ne reste plus qu'à écrire le produit final dans un fichier CSV à l'aide de l'opérateur `Write`.

In [15]:
csv_fp = f'./data/{sat_img_name}.csv'

In [16]:
write_csv_op = Operator('Write')
write_csv_op.formatName = 'CSV'
write_csv_op.file = csv_fp

On peut également écrire le produit final en GeoTIFF, toujours via l'opérateur `Write`.

In [17]:
geotiff_fp = f'./data/{sat_img_name}.tif'

In [18]:
write_tiff_op = Operator('Write')
write_tiff_op.formatName = 'GeoTIFF'
write_tiff_op.file = geotiff_fp

On peut maintenant créer le DAG associé. Tous les opérateurs que nous venons de définir seront des sommets de ce graphe là. Il reste à définir les arrêtes, c'est-à-dire spécifier l'ordre dans lequel ces opérations doivent s'exécuter. Cela se fait grâce au paramètre  `source` de la fonction `add_node`.

Voici le graphe que nous allons construire :

<img src="img/graph_c2rcc.png" alt="DAG décrivant les opérations à réaliser sur l'image satellite." style="width: 50%;"/>

In [19]:
G = Graph()

G.add_node(operator=read_op, node_id='read', source=None)
G.add_node(operator=resample_op, node_id='resample', source='read')
G.add_node(operator=subset_op, node_id='subset_img', source='resample')
G.add_node(operator=c2rcc_op, node_id='c2rcc', source='subset_img')

for node_id, op in bands_op.items():
    G.add_node(operator=op, node_id=node_id, source='c2rcc')

G.add_node(operator=band_merge_op, node_id='band_merge', source=list(bands_op.keys()) + ['c2rcc'])
G.add_node(operator=band_subset_op, node_id='subset_band', source='band_merge')
G.add_node(operator=write_csv_op, node_id='write_csv', source='subset_band')
G.add_node(operator=write_tiff_op, node_id='write_tiff', source='subset_band')

targetBandDescriptors <snapista.target_band_descriptors.TargetBandDescriptors object at 0x7fcc38eef220>
targetBandDescriptors <snapista.target_band_descriptors.TargetBandDescriptors object at 0x7fcc38eef5b0>
targetBandDescriptors <snapista.target_band_descriptors.TargetBandDescriptors object at 0x7fcc38eef670>
targetBandDescriptors <snapista.target_band_descriptors.TargetBandDescriptors object at 0x7fcc38eef4f0>
targetBandDescriptors <snapista.target_band_descriptors.TargetBandDescriptors object at 0x7fcc38eef940>
targetBandDescriptors <snapista.target_band_descriptors.TargetBandDescriptors object at 0x7fcc38eef2e0>
targetBandDescriptors <snapista.target_band_descriptors.TargetBandDescriptors object at 0x7fcc38eefb80>


In [20]:
G.nice_view()

In [21]:
G.run()

Processing the graph
Executing processing graph
INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
.10%..21%.31%.42%.52%.63%...74%..84%. done.
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: GDAL not found on system. Internal GDAL 3.0.0 from distribution will be used. (f1)


/home/baptiste/miniconda3/envs/ynov_ubuntu/lib/python3.8/site-packages/snapista-0.2.0-py3.8.egg/snapista/graph.py:422: ResourceWarning: unclosed file <_io.BufferedReader name=1555>
/home/baptiste/miniconda3/envs/ynov_ubuntu/lib/python3.8/site-packages/snapista-0.2.0-py3.8.egg/snapista/graph.py:422: ResourceWarning: unclosed file <_io.BufferedReader name=1558>


0

In [22]:
graph_fp = './data/example_graph.xml'
G.save_graph(graph_fp)

In [23]:
G_from_file = graph_io.read_file(graph_fp)

In [24]:
G_from_file.nice_view()

In [25]:
df = pd.read_csv(csv_fp, sep='\t', header=1)
df

,featureId,x:ushort,y:ushort,lat:float,lon:float,c2rcc_flags:float,conc_chl:float,unc_chl:float
0,0,0,0,46.959656,-71.406784,2.147484e+09,0.911549,0.888275
1,1,1,0,46.959633,-71.405990,2.147484e+09,0.001799,0.000363
2,2,2,0,46.959606,-71.405205,2.147484e+09,0.001814,0.000434
3,3,3,0,46.959583,-71.404420,2.147484e+09,0.001813,0.000406
4,4,4,0,46.959557,-71.403630,2.147500e+09,0.001805,0.000563
...,...,...,...,...,...,...,...,...
5147,5147,51,91,46.909355,-71.369965,2.147484e+09,0.636196,0.604002
5148,5148,52,91,46.909330,-71.369180,2.147484e+09,196.332100,195.779510
5149,5149,53,91,46.909306,-71.368385,2.147484e+09,1.250486,1.231746
5150,5150,54,91,46.909280,-71.367600,2.147486e+09,0.002476,0.000252


In [26]:
df['geometry'] = df[['lat:float', 'lon:float']].apply(lambda row: Point(row['lon:float'], row['lat:float']), axis=1)
gdf_csv_pixels = gpd.GeoDataFrame(df, crs='EPSG:4326', geometry='geometry')
gdf_csv_pixels.to_file(f'./data/{sat_img_name}_csv_all_pixels.geojson')

/home/baptiste/miniconda3/envs/ynov_ubuntu/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:130: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [27]:
gdf_csv_pixels = (gdf_csv_pixels.sjoin(gdf_lake, how='inner', predicate='within')
                  .drop(columns='index_right')
                  .reset_index(drop=True))

In [28]:
gdf_csv_pixels.to_file(f'./data/{sat_img_name}_csv_lake_pixels.geojson')

In [29]:
data = {}

with rasterio.Env():
    with rasterio.open(geotiff_fp) as src:
        crs = str(src.crs)
        all_bands = src.read()
        nb_bands = all_bands.shape[0]
        for i, band in enumerate(all_bands):
            data[bands[i]] = band.flatten()
        
        data['geometry'] = [Polygon(x[0]['coordinates'][0]) for x in rasterio.features.shapes(band, mask=None, transform=src.transform)] 

gdf_tiff = gpd.GeoDataFrame(pd.DataFrame(data=data), crs=crs, geometry='geometry').to_crs('EPSG:4326')
gdf_tiff.to_file(f'./data/{sat_img_name}_tiff_all_pixels.geojson')

In [30]:
gdf_tiff = (gdf_tiff.sjoin(gdf_lake, how='inner', predicate='within')
            .drop(columns='index_right')
            .reset_index(drop=True)
           )

gdf_tiff.to_file(f'./data/{sat_img_name}_tiff_lake_pixels.geojson')